<a href="https://colab.research.google.com/github/vgbarone/otca/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Instalando o SDK do Google
!pip install -q -U google-generativeai

In [4]:
# Configurações iniciais

from flask import Flask, render_template, request, jsonify, session
import google.generativeai as genai
import json
import datetime
import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
import secrets # Importe secrets para gerar chave secreta

app = Flask(__name__, template_folder='templates', static_folder='static')

# Gere uma chave secreta aleatória para as sessões do Flask
app.secret_key = secrets.token_urlsafe(16)

#GOOGLE API KEY
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

In [12]:
# Carrega o arquivo JSON com informações sobre a Lei de Crimes Ambientais
with open('/content/drive/MyDrive/Colab Notebooks/OTCA/lei_crimes_ambientais.json', 'r') as f:
    lei_data = json.load(f)

In [5]:
#Listando os modelos disponíveis
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [6]:
generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}

In [7]:
safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

In [8]:
model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)

In [9]:
chat = model.start_chat(history=[])

In [10]:
# Função para salvar a denúncia no Google Sheets
def salvar_denuncia(denuncia):
    auth.authenticate_user()
    gc = gspread.authorize(GoogleCredentials.get_application_default())

    # ID da planilha (substitua pelo ID da sua planilha)
    planilha_id = "1jbC2vKsy-Ku6MEraouV4LIM58LPb8f8s0eUpPWi7-J4" # Substitua pelo ID da sua planilha


    # Abre a planilha
    planilha = gc.open_by_key(planilha_id)

    # Seleciona a primeira aba da planilha
    aba = planilha.sheet1

    # Adiciona a denúncia à planilha
    aba.append_row([
        denuncia['data_hora'],
        denuncia['coordenadas'],
        denuncia['nome'],
        denuncia['email'],
        denuncia['endereco'],
        denuncia['uf'],
        denuncia['municipio'],
        denuncia['texto']
    ])

In [11]:
# Função para buscar informações na Lei de Crimes Ambientais
def encontrar_informacao_lei(pergunta):
    for item in lei_data:
        if any(keyword in pergunta.lower() for keyword in item['keywords']):
            return item['informacao']
    return None

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/enviar_mensagem', methods=['POST'])
def enviar_mensagem():
    mensagem = request.form['mensagem']

    # Processar a mensagem com o Google Gemini
    informacao_lei = encontrar_informacao_lei(mensagem)
    if informacao_lei:
        prompt_gemini = f"Aqui está uma pergunta sobre a Lei de Crimes Ambientais: {mensagem}\n\n Aqui está alguma informação relevante da lei: {informacao_lei}\n\n Por favor, responda a pergunta de forma informativa."
    else:
        prompt_gemini = f"Aqui está uma pergunta sobre a Lei de Crimes Ambientais: {mensagem}\n\n Por favor, responda a pergunta da melhor forma possível."

    response = chat.send_message(prompt_gemini)

    return jsonify({'resposta': response.text})

@app.route('/salvar_coordenadas', methods=['POST'])
def salvar_coordenadas():
    latitude = request.form['latitude']
    longitude = request.form['longitude']
    nome = request.form['nome']
    email = request.form['email']
    endereco = request.form['endereco']
    texto = request.form['texto']
    uf = request.form['uf']  # Obtem UF do formulário
    municipio = request.form['municipio']  # Obtem Município do formulário

    # Coleta data e hora automaticamente
    data_hora = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    coordenadas = f"{latitude}, {longitude}"

    # Criar dicionário da denúncia
    denuncia = {
        "data_hora": data_hora,
        "coordenadas": coordenadas,
        "nome": nome,
        "email": email,
        "endereco": endereco,
        "texto": texto,
        "uf": uf,  # Adiciona UF ao dicionário
        "municipio": municipio  # Adiciona Município ao dicionário
    }

    # Salvar denúncia no Google Sheets
    salvar_denuncia(denuncia)

    return jsonify({'status': 'success'})

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
